In [ ]:
import glob
import cv2 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import imutils
import copy
from scipy.ndimage.filters import  gaussian_filter
import math
import skimage.transform as sktr


### Loading images

In [ ]:
paths = [file for file in glob.glob('./inputs/P4/b/*.png')]
images = []
for path in paths:
    img = cv2.imread(path)
    images.append(img)
    

### Fourier

In [ ]:
def get_fourier(img):
    img = img.copy()
    img_dft = cv2.dft(np.float32(img),flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(img_dft)
    magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
    magnitude = np.log(magnitude)
    
    return magnitude, angle


def pattern_remover(img,p=0.65,center=25):
    img = img.copy()
    out1 = img.copy()
    dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    magnitude, angle = cv2.cartToPolar(dft_shift[:, :, 0],dft_shift[:, :, 1])
    magnitude = np.log(magnitude)
    out2 = magnitude.copy()

    _max = max(magnitude.flatten())

    for row in range(magnitude.shape[0]):
        for col in range(magnitude.shape[1]):
            if row > magnitude.shape[0]//2 - center and row < magnitude.shape[0]//2 + center and col > magnitude.shape[1]//2 - center and col < magnitude.shape[1]//2 + center :
                continue 

            if magnitude[row,col] >= _max*p :
                magnitude[row,col] = 0 

    out3 = magnitude.copy()
    i_magnitude = np.exp(magnitude)
    i_dft_shift = cv2.polarToCart(i_magnitude,angle)
    i_dft_shift = np.stack([i_dft_shift[0],i_dft_shift[1]],axis=2)
    i_dft_shift = np.fft.ifftshift(i_dft_shift)
    i_dft = cv2.idft(i_dft_shift, flags=cv2.DFT_SCALE)
    out4 = cv2.magnitude(i_dft[:,:,0],i_dft[:,:,1])
    
    return out1,out2,out3,out4
    


In [ ]:
Codes = [cv2.COLOR_BGR2HSV,cv2.COLOR_BGR2YCrCb,cv2.COLOR_BGR2YUV,cv2.COLOR_BGR2LAB,cv2.COLOR_BGR2RGB]
tags = ["HSV" , "YCrCb" , "YUV","LAB","RGB"]
plt.rcParams['figure.figsize'] = [8, 10]

for Index in range(3) :
    for i in range(5):
        img_tmp = cv2.cvtColor(images[Index],Codes[i])
        
        I = 1 
        for j in range(3):
            plt.subplot(3,2,I)
            I += 1
            plt.imshow(img_tmp[:,:,j],cmap="gray")
            plt.title("{2} | {0} Channel #{1}".format(tags[i],j+1,list(paths[Index].split("\\"))[-1]))
            plt.axis("off")
            
            plt.subplot(3,2,I)
            I += 1
            m,_ =get_fourier(img_tmp[:,:,j])
            plt.imshow(m,cmap="gray")
            plt.title("{0} Channel #{1} (magnitude)".format(tags[i],j+1))
            plt.axis("off")
            
        plt.tight_layout()
        plt.show()
            

### Cagi_1 proc

In [ ]:
Codes = [cv2.COLOR_BGR2LAB,None,None]
I_Codes = [cv2.COLOR_LAB2RGB,None,None]
Param = [ [[1,20],[0.585,20],[0.585,20]],None,None]
tags = ["LAB" ,None,None]
plt.rcParams['figure.figsize'] = [16, 10]

for Index in range(0,1) :
    img_bgr = images[Index].copy()
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")


    plt.show()
    break
    

### Cagi_2 proc

In [ ]:
Codes = [None,cv2.COLOR_BGR2LAB,None]
I_Codes = [None,cv2.COLOR_LAB2RGB,None]
Param = [None, [[0.7,20],[0.48,20],[0.48,20]],None]
tags = [None,"LAB" ,None]
plt.rcParams['figure.figsize'] = [16, 10]

for Index in range(1,2) :
    img_bgr = images[Index].copy()
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")


    plt.show()
    break
    

### Cagi_3 proc

In [ ]:
Codes = [None,None,cv2.COLOR_BGR2LAB]
I_Codes = [None,None,cv2.COLOR_LAB2RGB]
Param = [None,None, [[1,20],[0.55,20],[0.55,20]]]
tags = [None,None,"LAB"]
plt.rcParams['figure.figsize'] = [16, 10]

for Index in range(2,3) :
    img_bgr = images[Index].copy()
    img_s = cv2.cvtColor(img_bgr,Codes[Index])
    para = Param[Index]
    
    P_index = 1
    for i in range(3):
        _p,_center = para[i]
        outs = pattern_remover(img_s[:,:,i],_p,_center)
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[0],cmap="gray")
        plt.title("{0} Channel #{1}".format(tags[Index],i+1))
        plt.axis("off")
    
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[1],cmap="gray")
        plt.title("{0} Channel #{1} magnitude".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[2],cmap="gray")
        plt.title("{0} Channel #{1} magnitude after remove".format(tags[Index],i+1))
        plt.axis("off")
        
        plt.subplot(3,4,P_index)
        P_index += 1
        plt.imshow(outs[3],cmap="gray")
        plt.title("{0} Channel #{1} after remove".format(tags[Index],i+1))
        plt.axis("off")
        img_s[:,:,i] = outs [3]
        
    plt.tight_layout()
    plt.show()
    
    plt.rcParams['figure.figsize'] = [16,8]
    img_s = cv2.cvtColor(img_s,I_Codes[Index])
    plt.subplot(1,2,1)
    plt.imshow(img_bgr[:,:,::-1])
    plt.title("Input Image")
    plt.axis("off")
    
    plt.subplot(1,2,2)
    plt.imshow(img_s)
    plt.title("Filtered Image")
    plt.axis("off")


    plt.show()
    break
    